In [1]:
import pandas as pd
import html_parser as htmlParser
import re

In [2]:
try:
    Answers = pd.read_csv('Answers.csv')
except:
    Answers = pd.read_csv("Answers.csv",encoding = "ISO-8859-1")

In [3]:
parser2 = htmlParser.Parser()

In [4]:
def extract_html(data):
    parser2.feed(data)
    result = " ".join(htmlParser.text_data)
    htmlParser.text_data = []
    return result

In [5]:
Answers["Content"] = Answers["Body"].apply(lambda x:extract_html(x))

In [6]:
Answers = Answers.drop(columns = ["Body","Id","OwnerUserId","CreationDate"])

In [7]:
Answers.head()

,ParentId,Score,Content
0,90,13,Version Control with Subversion \r\n\r\n A ver...
1,80,12,"I wound up using this. It is a kind of a hack,..."
2,180,1,I've read somewhere the human eye can't distin...
3,260,4,"Yes, I thought about that, but I soon figured ..."
4,260,28,Oleg Shilo's C# Script solution (at The Code P...


In [33]:
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords

In [58]:
temp = Answers.iloc[1,-1]
print(temp)
s = re.sub(r'[^\w\s]','',temp)
escapes = '\b\n\r\t\\'
for c in escapes:
    s = s.replace(c,"")
s

I wound up using this. It is a kind of a hack, but it actually works pretty well. The only thing is you have to be very careful with your semicolons. : D 

 var strSql:String = stream.readUTFBytes(stream.bytesAvailable);      
var i:Number = 0;
var strSqlSplit:Array = strSql.split(";");
for (i = 0; i < strSqlSplit.length; i++){
    NonQuery(strSqlSplit[i].toString());
}
 



'I wound up using this It is a kind of a hack but it actually works pretty well The only thing is you have to be very careful with your semicolons  D  var strSqlString  streamreadUTFBytesstreambytesAvailable      var iNumber  0var strSqlSplitArray  strSqlsplitfor i  0 i  strSqlSplitlength i    NonQuerystrSqlSplititoString '

In [35]:
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\beboz\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [23]:

# WordNet POS tags are: NOUN = 'n', ADJ = 's', VERB = 'v', ADV = 'r', ADJ_SAT = 'a'
# Descriptions (c) https://web.stanford.edu/~jurafsky/slp3/10.pdf
tag_map = {
        'CC':None, # coordin. conjunction (and, but, or)  
        'CD':wn.NOUN, # cardinal number (one, two)             
        'DT':None, # determiner (a, the)                    
        'EX':wn.ADV, # existential ‘there’ (there)           
        'FW':None, # foreign word (mea culpa)             
        'IN':wn.ADV, # preposition/sub-conj (of, in, by)   
        'JJ':wn.ADJ, # adjective (yellow)                  
        'JJR':wn.ADJ,  # adj., comparative (bigger)          
        'JJS':wn.ADJ,  # adj., superlative (wildest)           
        'LS':None, # list item marker (1, 2, One)          
        'MD':None, # modal (can, should)                    
        'NN':wn.NOUN, # noun, sing. or mass (llama)          
        'NNS':wn.NOUN, # noun, plural (llamas)                  
        'NNP':wn.NOUN, # proper noun, sing. (IBM)              
        'NNPS':wn.NOUN, # proper noun, plural (Carolinas)
        'PDT':wn.ADJ, # predeterminer (all, both)            
        'POS':None, # possessive ending (’s )               
        'PRP':None, # personal pronoun (I, you, he)     
        'PRP$':None, # possessive pronoun (your, one’s)    
        'RB':wn.ADV, # adverb (quickly, never)            
        'RBR':wn.ADV, # adverb, comparative (faster)        
        'RBS':wn.ADV, # adverb, superlative (fastest)     
        'RP':[wn.ADJ, wn.ADJ_SAT], # particle (up, off)
        'SYM':None, # symbol (+,%, &)
        'TO':None, # “to” (to)
        'UH':None, # interjection (ah, oops)
        'VB':wn.VERB, # verb base form (eat)
        'VBD':wn.VERB, # verb past tense (ate)
        'VBG':wn.VERB, # verb gerund (eating)
        'VBN':wn.VERB, # verb past participle (eaten)
        'VBP':wn.VERB, # verb non-3sg pres (eat)
        'VBZ':wn.VERB, # verb 3sg pres (eats)
        'WDT':None, # wh-determiner (which, that)
        'WP':None, # wh-pronoun (what, who)
        'WP$':None, # possessive (wh- whose)
        'WRB':None, # wh-adverb (how, where)
        '$':None, #  dollar sign ($)
        '#':None, # pound sign (#)
        '“':None, # left quote (‘ or “)
        '”':None, # right quote (’ or ”)
        '(':None, # left parenthesis ([, (, {, <)
        ')':None, # right parenthesis (], ), }, >)
        ',':None, # comma (,)
        '.':None, # sentence-final punc (. ! ?)
        ':':None # mid-sentence punc (: ; ... – -)
    }

In [68]:
lemmatizer=WordNetLemmatizer()
def lem(text):
    global iteration
    iteration += 1
    text = text.lower()
    token=word_tokenize(text)
    tags=pos_tag(token)
    w_net_lemma=[]
    for tag in tags:
        try:
            w_net_lemma.append(lemmatizer.lemmatize(tag[0],pos=tag_map[tag[1]]))
        except:
            continue
    if iteration % 10000 == 0:
        print("Iteration: "+ str(iteration))
    return " ".join(w_net_lemma)

In [64]:
stop_words = stopwords.words('english')
stop_words.extend(["'re","'s","'m"])
lemmatizer = WordNetLemmatizer()
iteration = 0
def clean_text(text):
    global iteration
    iteration += 1
    text = nltk.word_tokenize(text)
    clean_text = []
    lemma_text = []
    for c in text:
        if c not in stop_words:
            clean_text.append(c)
            lemma_text.append(lemmatizer.lemmatize(c))
    if iteration % 1000 == 0:
        print("Iteration: "+ str(iteration))
    return " ".join(lemma_text)

In [27]:
print(lem(temp))
print(s)

i 'm guess that because want set for user that 're on kind of share computer be probably run under domain here be dragon let say joe jane regularly log onto computer then have 'registries then install app installer employ giant hack disgust thing set item under hkcu for then bob come along log on 500 other people have account in domain so do 's never use computer before so have registry first time log in window create one n't have set app then fall over behaves incorrectly bob complains loudly about crappy product from raynixon incorporate correct answer be just have default setting in app write registry if do n't find 's general good practice that app never depend on registry create thing as need for registry entry not just hkcu anyway
Im guessing that because you want to set it for all users that youre on some kind of shared computer which is probably running under a domain  HERE BE DRAGONS  Lets say Joe and Jane regularly log onto the computer then they will each have registries  Yo

In [12]:
# Answers["Content"] = Answers["Content"].apply(lambda x : lem(x))

In [59]:
clean_text(temp.lower())
lem(temp)

wound using . kind hack , actually works pretty well . thing careful semicolons . : var strsql : string = stream.readutfbytes ( stream.bytesavailable ) ; var : number = 0 ; var strsqlsplit : array = strsql.split ( `` ; '' ) ; ( = 0 ; < strsqlsplit.length ; i++ ) { nonquery ( strsqlsplit [ ] .tostring ( ) ) ; }
wound using . kind hack , actually work pretty well . thing careful semicolon . : var strsql : string = stream.readutfbytes ( stream.bytesavailable ) ; var : number = 0 ; var strsqlsplit : array = strsql.split ( `` ; '' ) ; ( = 0 ; < strsqlsplit.length ; i++ ) { nonquery ( strsqlsplit [ ] .tostring ( ) ) ; }


'i wind use be kind of hack actually work pretty well only thing be have be very careful with semicolon d var strsql string = stream.readutfbytes stream.bytesavailable i number = 0 var strsqlsplit array = strsql.split for i = 0 i < strsqlsplit.length i++ nonquery strsqlsplit [ i ] .tostring'

I'm guessing that because you want to set it for all users, that you're on some kind of shared computer, which is probably running under a domain? 

 HERE BE DRAGONS 

 Let's say Joe and Jane regularly log onto the computer, then they will each have 'registries'. 

 You'll then install your app, and the installer will employ giant hacks and disgusting things to set items under HKCU for them. 

 THEN, bob will come along and log on (he, and 500 other people have accounts in the domain and so can do this). He's never used this computer before, so he has no registry. The first time he logs in, windows creates him one, but he won't have your setting.  

 Your app then falls over or behaves incorrectly, and bob complains loudly about those crappy products from raynixon incorporated. 

 The correct answer is to just have some default settings in your app, which can write them to the registry if it doesn't find them. It's general good practice that your app should never depend on the registry

In [65]:
temp = Answers["Content"].apply(lambda x: clean_text(x))

Iteration: 1000
Iteration: 2000
Iteration: 3000
Iteration: 4000
Iteration: 5000
Iteration: 6000
Iteration: 7000
Iteration: 8000
Iteration: 9000
Iteration: 10000
Iteration: 11000
Iteration: 12000
Iteration: 13000
Iteration: 14000
Iteration: 15000
Iteration: 16000
Iteration: 17000
Iteration: 18000
Iteration: 19000
Iteration: 20000
Iteration: 21000
Iteration: 22000
Iteration: 23000
Iteration: 24000
Iteration: 25000
Iteration: 26000
Iteration: 27000
Iteration: 28000
Iteration: 29000
Iteration: 30000
Iteration: 31000
Iteration: 32000
Iteration: 33000
Iteration: 34000
Iteration: 35000
Iteration: 36000
Iteration: 37000
Iteration: 38000
Iteration: 39000
Iteration: 40000
Iteration: 41000
Iteration: 42000
Iteration: 43000
Iteration: 44000
Iteration: 45000
Iteration: 46000
Iteration: 47000
Iteration: 48000
Iteration: 49000
Iteration: 50000
Iteration: 51000
Iteration: 52000
Iteration: 53000
Iteration: 54000
Iteration: 55000
Iteration: 56000
Iteration: 57000
Iteration: 58000
Iteration: 59000
Iterat

Iteration: 463000
Iteration: 464000
Iteration: 465000
Iteration: 466000
Iteration: 467000
Iteration: 468000
Iteration: 469000
Iteration: 470000
Iteration: 471000
Iteration: 472000
Iteration: 473000
Iteration: 474000
Iteration: 475000
Iteration: 476000
Iteration: 477000
Iteration: 478000
Iteration: 479000
Iteration: 480000
Iteration: 481000
Iteration: 482000
Iteration: 483000
Iteration: 484000
Iteration: 485000
Iteration: 486000
Iteration: 487000
Iteration: 488000
Iteration: 489000
Iteration: 490000
Iteration: 491000
Iteration: 492000
Iteration: 493000
Iteration: 494000
Iteration: 495000
Iteration: 496000
Iteration: 497000
Iteration: 498000
Iteration: 499000
Iteration: 500000
Iteration: 501000
Iteration: 502000
Iteration: 503000
Iteration: 504000
Iteration: 505000
Iteration: 506000
Iteration: 507000
Iteration: 508000
Iteration: 509000
Iteration: 510000
Iteration: 511000
Iteration: 512000
Iteration: 513000
Iteration: 514000
Iteration: 515000
Iteration: 516000
Iteration: 517000
Iteration:

Iteration: 919000
Iteration: 920000
Iteration: 921000
Iteration: 922000
Iteration: 923000
Iteration: 924000
Iteration: 925000
Iteration: 926000
Iteration: 927000
Iteration: 928000
Iteration: 929000
Iteration: 930000
Iteration: 931000
Iteration: 932000
Iteration: 933000
Iteration: 934000
Iteration: 935000
Iteration: 936000
Iteration: 937000
Iteration: 938000
Iteration: 939000
Iteration: 940000
Iteration: 941000
Iteration: 942000
Iteration: 943000
Iteration: 944000
Iteration: 945000
Iteration: 946000
Iteration: 947000
Iteration: 948000
Iteration: 949000
Iteration: 950000
Iteration: 951000
Iteration: 952000
Iteration: 953000
Iteration: 954000
Iteration: 955000
Iteration: 956000
Iteration: 957000
Iteration: 958000
Iteration: 959000
Iteration: 960000
Iteration: 961000
Iteration: 962000
Iteration: 963000
Iteration: 964000
Iteration: 965000
Iteration: 966000
Iteration: 967000
Iteration: 968000
Iteration: 969000
Iteration: 970000
Iteration: 971000
Iteration: 972000
Iteration: 973000
Iteration:

In [69]:
temp2 = Answers["Content"].apply(lambda x: lem(x))

Iteration: 10000
Iteration: 20000
Iteration: 30000
Iteration: 40000
Iteration: 50000
Iteration: 60000
Iteration: 70000
Iteration: 80000
Iteration: 90000
Iteration: 100000
Iteration: 110000
Iteration: 120000
Iteration: 130000
Iteration: 140000
Iteration: 150000
Iteration: 160000
Iteration: 170000
Iteration: 180000
Iteration: 190000
Iteration: 200000
Iteration: 210000
Iteration: 220000
Iteration: 230000
Iteration: 240000
Iteration: 250000
Iteration: 260000
Iteration: 270000
Iteration: 280000
Iteration: 290000
Iteration: 300000
Iteration: 310000
Iteration: 320000
Iteration: 330000
Iteration: 340000
Iteration: 350000
Iteration: 360000
Iteration: 370000
Iteration: 380000
Iteration: 390000
Iteration: 400000
Iteration: 410000
Iteration: 420000
Iteration: 430000
Iteration: 440000
Iteration: 450000
Iteration: 460000
Iteration: 470000
Iteration: 480000
Iteration: 490000
Iteration: 500000
Iteration: 510000
Iteration: 520000
Iteration: 530000
Iteration: 540000
Iteration: 550000
Iteration: 560000
I

In [66]:
iteration = 0

In [71]:
Answers["Content"] = temp2

In [72]:
Answers.head()

,ParentId,Score,Content
0,90,13,version control with subversion very good reso...
1,80,12,i wind use be kind of hack actually work prett...
2,180,1,i 've read somewhere human eye n't distinguish...
3,260,4,i think about i soon figure that domain-specif...
4,260,28,oleg shilo c script solution at code project r...


In [76]:
Answers = Answers.drop_duplicates()

In [77]:
Answers

,ParentId,Score,Content
0,90,13,version control with subversion very good reso...
1,80,12,i wind use be kind of hack actually work prett...
2,180,1,i 've read somewhere human eye n't distinguish...
3,260,4,i think about i soon figure that domain-specif...
4,260,28,oleg shilo c script solution at code project r...
...,...,...,...
1048570,19483190,0,answer be initially create own acl allow acces...
1048571,19505410,2,use pygame.key.set_repeat disable repeated eve...
1048572,19506290,1,create table table1 id varchar 10 tid int stat...
1048573,19493260,1,finally be i end up with i hope help consider ...


In [78]:
Answers.to_csv("Preprocessed_answers.csv")

In [79]:
try:
    Answers2 = pd.read_csv('Answers.csv')
except:
    Answers2 = pd.read_csv("Answers.csv",encoding = "ISO-8859-1")

In [82]:
Answers2 = pd.read_csv('Preprocessed_answers.csv')